# Verification

For verification, we will test our models on comments from the month of November 2015. In a similar manner as we scraped, we will take top-level comments from the top reddit threads of that month and predict scores.

In [1]:
import time

mmddyy_range = ('11-01-2015', '11-30-2015')
f_epoch = lambda x: int(time.mktime(time.strptime(x, '%m-%d-%Y')))
time_range = str(f_epoch(mmddyy_range[0])) + '..' + str(f_epoch(mmddyy_range[1]))
print time_range

1446350400..1448859600


We will thus scrape from the URL https://www.reddit.com/r/AskReddit/search?sort=top&q=timestamp%3A1446350400..1448859600&restrict_sr=on&syntax=cloudsearch, where we sort by "top" this time instead of "comments" -- we want threads that have been voted extensively upon. The result should be the top-level comments scraped from the highest-voted twenty-five threads.

In [2]:
url = 'https://www.reddit.com/r/AskReddit/search?sort=top&q=timestamp%3A1446350400..1448859600&restrict_sr=on&syntax=cloudsearch'

In [3]:
from bs4 import BeautifulSoup
import urllib2
import re

data = BeautifulSoup(urllib2.urlopen(url).read(), 'html.parser')

In [4]:
rows = data.findAll("a", attrs = {'class': 'search-comments may-blank'}, limit=None)
url_list = [str(r.attrs['href'])[:-17] + '.json' for r in rows]

thread_ids = []
for elt in url_list:
    time.sleep(30)
    json_text = urllib2.urlopen(elt).read()
    thread_ids.append(re.search(r'\"id\": \"([a-zA-Z0-9_]+)\"', json_text).group(1))

['3tdwy2', '3ucqxj', '3regbo', '3r2siv', '3secjr', '3ta5bz', '3si0bb', '3t1q82', '3tfpe2', '3ruhnj', '3r76bx', '3so18a', '3rbzln', '3tkemy', '3rih54', '3u2szy', '3u76qe', '3su6er', '3tid38', '3tqjr6', '3ur7jx', '3rg1rf', '3ualrk', '3ss8oz', '3rmbnh']


In [5]:
import praw
r = praw.Reddit('Getting comments for CS109 Final Project')

thread_d = {}
for t_id in thread_ids:
    time.sleep(10) # lots of HTTP request errors!
    submission = r.get_submission(submission_id=t_id)
    submission.replace_more_comments(limit=100, threshold=0)
    all_comments = submission.comments
    thread_d[t_id] = all_comments

In [6]:
print len(thread_d)

25


In [8]:
print sum([len(comments) for thread, comments in thread_d.items()])

36787


In [10]:
import cPickle
cPickle.dump(thread_d, open('v_comments_dict.p', 'wb')) 
thread_d_load = cPickle.load(open('v_comments_dict.p', 'rb'))
print len(thread_d_load)
print sum([len(comments) for thread, comments in thread_d_load.items()])

25
36787
